In [1]:
import numpy as np
import re
import pickle
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import load_files
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rohan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# txt_sentoken/ will loop through the sub directories and assign classes 0 and 1
reviews = load_files('txt_sentoken/')

In [3]:
# X will contain the data whereas y will contain the corresponding label
X, y = reviews.data, reviews.target

In [4]:
# persisting the data
# storing X and y as pickle file (for loading faster in case of large files)
with open('X.pickle', 'wb') as f:
    pickle.dump(X,f)

with open('y.pickle', 'wb') as f:
    pickle.dump(y,f)   

In [5]:
# Unpickling the dataset
with open('X.pickle', 'rb') as f:
    X = pickle.load(f)

with open('y.pickle', 'rb') as f:
    y = pickle.load(f)


In [6]:
# preprocessing the data
corpus = []
for i in range(len(X)):
    review = re.sub(r'\W',' ',str(X[i])) # substituting all the non-letter characters like punctuations, ascii with a space
    review = review.lower()
    review = re.sub(r'\s+[a-z]\s+',' ',review) # substituting all the single character words with space as they are not so importan, genrellay they are preceded and succeded by a space
    review = re.sub(r'^[a-z]\s+',' ',review) # also substituting sinle character words from the start of the sentence
    review = re.sub(r'\s+',' ',review) # substituting the extra spaces generated with a single space
    corpus.append(review)

In [7]:
# creating the bag of words model
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features = 2000, min_df = 3, max_df = 0.6, stop_words = stopwords.words('english'))
# max_fratures: 2000 most frequent words as feature
# min_df: exclude all those words which appear in three or less than three documents
# max_df: (specified in percentage) exclude all those words which appears in more than 60% of the document
# stop_words: exclude all the stop words in the document

X = vectorizer.fit_transform(corpus).toarray()

In [8]:
# transforming the bag of words model to tf-idf model
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
X = transformer.fit_transform(X).toarray()

In [9]:
# splitting the data into train and test set
from sklearn.model_selection import train_test_split
text_train, text_test, sent_train, sent_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [10]:
# building our logistic regression classifier amd training the train-dataset
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(text_train, sent_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [11]:
# predicting and finding the accuracy score
from sklearn.metrics import accuracy_score
pred = clf.predict(text_test)
accuracy = accuracy_score(sent_test, pred)
print("accuracy:", accuracy)

accuracy: 0.8475
